# Understanding the performance of the vecadd kernel


In this section we will see how we can avoid the JIT cost of Numba, how we can measure the performance of the kernel without the `%timeit` magic, how we can use `nvprof`, the CUDA profiler to analyze the performance of the kernel, and finally, we will evaluate the performance of the kernel.

## Avoiding the JIT cost

Unlike the `@numba.jit` decorator, `@cuda.jit` does not accept a `cache` parameter, that would cache the generated code on the disk and use it on subsequent invocations of the program. Nonetheless, we can force the code generation at import time by supplying a function signature to the `@cuda.jit` decorator that describes the CUDA kernel. This will generate the CUDA code at the time when the decorator processes the function declaration and, therefore, we will avoid the runtime cost of JIT. Let's see how this is done:

In [1]:
import numba.cuda as cuda
import numpy as np


@cuda.jit('void(Array(float64, 1, "C"), Array(float64, 1, "C"), Array(float64, 1, "C"))')

def _vecadd_cuda(z, x, y):
    '''The CUDA kernel'''
    i = cuda.grid(1)
    N = x.shape[0]
    if i >= N:
        return

    z[i] = x[i] + y[i]


This instructs the Numba runtime to compile the following function into a CUDA kernel (return type `void`) accepting three one-dimensional arrays of `float64` (or `double`) stored in row-major order (C convention). This way, Numba does not have to wait until the `_vecadd_cuda` function is called to figure out the argument types and compile the kernel. It can do this at import time, when it first encounters the function. The downside to that is that you can't call the function with a different type of arguments later. For more details on how to specify function signatures in Numba, see [here](http://numba.pydata.org/numba-doc/latest/reference/types.html#numba-types).

Let's retry our example now with this version of the kernel.

In [2]:
# Set up the host vectors
N = 200*1000*100
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
_vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

#%timeit -n1 -r1

## Measuring the execution time of the kernel

All you see from the previous exercise is the same execution time! What is happening? Actually, you are not measuring the kernel execution time, but rather the kernel launch time. CUDA kernels are launched asynchronously. This means that as soon as you launch the kernel on the GPU, the CPU will continue execution. In this case, it will continue executing and it will block at the statement that copies back the result to the host. 

How do we measure the kernel execution time then? For this, we are going to write a Python [context manager](https://docs.python.org/3.8/reference/datamodel.html?highlight=__getitem__#with-statement-context-managers) so as to measure the execution time of a region in a nice, Pythonic way:

In [3]:
import time


class time_region:
    def __init__(self, time_offset=0):
        self._time_off = time_offset

    def __enter__(self):
        self._t_start = time.time()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._t_end = time.time()

    def elapsed_time(self):
        return self._time_off + (self._t_end - self._t_start)

Let's use our timer to time the kernel:

In [4]:
# Set up the host vectors
N = 200*100*100
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')

# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

CUDA kernel time: 0.0 s


Our timer seems to work fine; we still measure the kernel launch time as with `%timeit`. In order to measure the actual kernel execution time, we need to block the CPU calling thread until the kernel finishes, immediately after we launch the kernel. We can achieve that with `cuda.synchronize()`:

In [5]:
# Set up the host vectors
N = 200*100*100
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)
    cuda.synchronize()

with time_region() as t_ref:
    z = x + y

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')
print(f'Numpy time:       {t_ref.elapsed_time()} s')


# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

CUDA kernel time: 0.0 s
Numpy time:       0.015631437301635742 s


Not bad, the CUDA kernel is 15x faster than the native Numpy kernel.

Before analysing how good or bad this is, let's see an alternative way for measuring the kernel time that actually avoids the use of `cuda.synchronize()`.

## Measuring the kernel execution time with CUDA events

Inserting `cuda.synchronize()` without a reason could slow down your application, since it not only blocks the current CPU thread, but also imposes a synchronization point for all the CUDA streams on the GPU that are currently running in parallel.

A CUDA stream is essentially a series of sequential operations (data transfers, kernel launches, etc.) that execute on the GPU. Multiple CUDA streams may run independently on the GPU, thus allowing overlapping of operations, such as data transfers and execution of kernels.

To avoid this, but also to obtain a more precise measurement, we can use [CUDA events](http://docs.nvidia.com/cuda/cuda-c-programming-guide/#events).

Let's see how we can adapt our `time_region` context manager to use CUDA events:

In [6]:
class time_region_cuda:
    def __init__(self, time_offset=0, cuda_stream=0):
        self._t_start = cuda.event(timing=True)
        self._t_end = cuda.event(timing=True)
        self._time_off = time_offset
        self._cuda_stream = cuda_stream

    def __enter__(self):
        self._t_start.record(self._cuda_stream)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._t_end.record(self._cuda_stream)
        self._t_end.synchronize()

    def elapsed_time(self):
        return self._time_off + 1.e-3*cuda.event_elapsed_time(self._t_start, self._t_end)

To measure a data region with CUDA events we first need to create two events: one for the start and one for the end. We can achieve that with the `cuda.event(timing=True)`. To start counting, we need to call `record()` on the starting event marking the "arrival" to that milestone. Similarly, call the `record()` on the ending event to mark the end of the region. Then we can obtain the elapsed time using the corresponding function as shown in the example above.

Let's rewrite our vector addition example using the CUDA event timers:

In [7]:
# Set up the host vectors
N = 200*100*100
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region_cuda() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

with time_region() as t_ref:
    z = x + y

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')
print(f'Numpy time:       {t_ref.elapsed_time()} s')


# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

CUDA kernel time: 0.003547424077987671 s
Numpy time:       0.01562356948852539 s


As we can see the execution time obtained is the correct one without having to use `cuda.synchronize()`.

## Assessing the performance of the kernel

The question that arises is how good is the performance that we achieve. Let's inspect further the kernel. Each thread does two `float64` reads from the memory and one write and performs an addition. That means for one floating operation, the kernel must transfer to/from memory 24 bytes from the main memory. This gives us an  *arithmetic intensity* or *flop:byte ratio* of 0.0417. The lower this ratio is for a computational kernel, the more likely will be that the kernel is memory bandwidth bound. As the ratio increases, the kernel tends to be more compute bound. The theory behind the arithmetic intensity is covered by the *Roofline* performance model. For the moment, let's compute two performance metrics, the `Gflop/s` achieved by the kernel and the data transfer rate to/from memory:

In [8]:
print(f'Performance: {1.e-9*N/t_kernel.elapsed_time()} Gflop/s')
print(f'Transfer rate: {1.e-9*3*N*8/t_kernel.elapsed_time()} GB/s')

Performance: 0.5637893739320081 Gflop/s
Transfer rate: 13.530944974368197 GB/s


Let's compute the same for the NumPy kernel:

In [9]:
print(f'Performance: {1.e-9*N/t_ref.elapsed_time()} Gflop/s')
print(f'Transfer rate: {1.e-9*3*N*8/t_ref.elapsed_time()} GB/s')

Performance: 0.12801171982298185 Gflop/s
Transfer rate: 3.0722812757515645 GB/s


The GPU can deliver more than 10x bandwidth compared to the CPU. 

Achieving the nominal peak memory bandwidth is usually not possible with real-life computational kernels, even with very low arithmetic intensity. For this reason, we tend to cite the *effective memory bandwidth*, which is obtained by benchmarks.

The numpy vector addition performance is not ideal, since it can't reach the memory bandwidth limit as it ought to. The problem could be related to CPU affinity issues, but we are not going to address them in this tutorial.

## Understanding the data transfer overhead

So far we have only focused on the performance of the kernel. There is still a quite important topic we have not yet addressed. CUDA kernels require that the data they operate on is located on the device and we need to move that data there from the host. What is the cost of this data movement?

Let's time our benchmark code:

In [10]:
# Set up the host vectors
N = 200*100*100
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
with time_region_cuda() as t_copyin:
    d_x = cuda.to_device(x)
    d_y = cuda.to_device(y)

with time_region_cuda() as t_create:
    d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region_cuda() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

with time_region() as t_ref:
    z = x + y

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')
print(f'Numpy time:       {t_ref.elapsed_time()} s')


# Copy back the result to the host
with time_region_cuda() as t_copyout:
    res = d_z.copy_to_host()

print(f'Copyin time:  {t_copyin.elapsed_time()}')  
print(f'Create time:  {t_create.elapsed_time()}')    
print(f'Copyout time: {t_copyout.elapsed_time()}')    


# Validate the result
assert np.allclose(x + y, res)

CUDA kernel time: 0.00591318416595459 s
Numpy time:       0.015633821487426758 s
Copyin time:  0.03332368087768555
Create time:  3.103999886661768e-06
Copyout time: 0.016734304428100585


The data copy times are quite important! In fact, if we include these in the total execution time of the GPU version, the CPU version becomes more than 8x faster! Data transfers is the No. 1 optimization that we should do when programming for the GPUs. We must minimize the data transfers to/from GPU by keeping the necessary data on the GPU for as long as it is needed.

Now, let's see how fast is the data moved over to the GPU:

In [11]:
print(f'Copyin rate: {1e-9*2*N*8/t_copyin.elapsed_time()} GB/s')

Copyin rate: 0.9602780712447668 GB/s


This is bound by the data rate of the PCI 16x bus where the GPU is attached to and, it is indeed way too slower than the main memory bandwidth of modern processors.

Interestingly, the copyout data rate seems to be much slower:

In [12]:
print(f'Copyout rate: {1e-9*N*8/t_copyout.elapsed_time()} GB/s')

Copyout rate: 0.9561198117761306 GB/s


By default, memory allocated on the host is pageable. This means that it can be moved by the OS kernel to a secondary storage device if there is not enough memory available on the system. This can incur a significant performance penalty, especially if we write on freshly allocated memory (as it happens in this example). We can avoid this overhead by using *page-locked* or *pinned* memory. This memory cannot be paged out and it is physically resident on the memory device. CUDA gives you the opportunity to use pinned memory and Numba allows you to create pinned ndarrays using the [cuda.pinned_array()](http://numba.pydata.org/numba-doc/latest/cuda-reference/memory.html#numba.cuda.pinned_array) function.

In order to keep track of which memory pages are resident on the physical memory and which are not, the OS kernel maintains a special data structure called *page table*. When we allocate memory on the host, the OS kernel simply creates a virtual memory mapping and it does not allocate any physical page. As soon as we start writing to the memory area you have been allocated, it will look for the page in its page tables and if not found, a *page fault* will be raised and then the kernel will have to physically allocate the missing memory page and update its page tables.

Let's rewrite the copyout part using pinned memory:

In [13]:
res = cuda.pinned_array(N)
with time_region_cuda() as t_pinned:
    d_z.copy_to_host(res)
    
assert np.allclose(x + y, res)
print(f'Copyout data rate (pinned): {1e-9*N*8/t_pinned.elapsed_time()} GB/s')

Copyout data rate (pinned): 1.5243483737646457 GB/s


Notice how much the performance has improved. It is now even better than the copyin operation.

However, pinned memory does not come without a cost. Since pinned pages cannot be paged out, they will stay on the physical memory, increasing the memory pressure and, finally, the effective memory consumption of the code. For memory hungry applications, this can be a problem.